# Notebook for IBM capstone Project 

In [24]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 


# Data Preprocessing 

In [36]:
#Importing the table from the wiki page
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.columns = ['Postal Code','Borough','Neighborhood']
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [37]:
#Dropping rows that are not associated to any Borough 
NA = df[df['Borough']=='Not assigned'].index
df.drop(NA,inplace=True)
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [38]:
#Checking to see if any Borough are given to not assigned neighbourhoods 
neigh = df[df['Neighborhood']=='Not assigned']
neigh

,Postal Code,Borough,Neighborhood


In [39]:
#4Square API Login 
CLIENT_ID = 'TSVZ0QHSX0QDIVJTP4L3O4CNFSSU2WP0PENXI1F04502OIHK' # your Foursquare ID
CLIENT_SECRET = 'BPRI1ROATI03W15Y5KQCQMYZ0C2TS4LWWJCL0IWGNG32IIY2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [40]:
df_coords = pd.read_csv('http://cocl.us/Geospatial_data')
df_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [44]:
df_pos = pd.merge(df, df_coords, on=['Postal Code'], how='inner')

df_tor = df_pos[['Borough', 'Neighborhood', 'Postal Code', 'Latitude', 'Longitude']].copy()

df_tor

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,Regent Park / Harbourfront,M5A,43.654260,-79.360636
3,North York,Lawrence Manor / Lawrence Heights,M6A,43.718518,-79.464763
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...
98,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,M8X,43.653654,-79.506944
99,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,East Toronto,Business reply mail Processing CentrE,M7Y,43.662744,-79.321558
101,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,M8Y,43.636258,-79.498509


In [49]:
df_tor.shape

(103, 5)

In [45]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [47]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto